# Practical 7 - Part 2A - Zuohe Zheng
This project explores the geometry of a single camera. The aim is to take several points on
a plane, and predict where they will appear in the camera image. Based on these observed
points, we will then try to re-estimate the Euclidean transformation relating the plane and
the camera. In practical 2B we will use this code to draw a wireframe cube
on an augmented reality marker.   You should use this
template for your code and fill in the missing sections marked "TO DO"

# Import libraries 

In [2]:
%matplotlib inline
import os 
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio

# TO DO - fill in projectiveCamera and estimatePlanePose functions  (you will have to utilise your solutions for solveAXEqualsZero and calcBestHomography from Part 1)

In [3]:
#The goal of this function is to project points in XCart through projective camera
#defined by intrinsic matrix K and extrinsic matrix T.
def projectiveCamera(K,T,XCart):
    
    # TO DO: Replace this
    # XImCart =

    # TO DO: Convert Cartesian 3d points XCart to homogeneous coordinates XHom
    # by appending a row of 1 to the original point coordinates to form [u,v,w,1]
    XHom = np.concatenate((XCart, np.ones((1,XCart.shape[1]))), axis=0)
    # TO DO: Apply extrinsic matrix to XHom, to move to frame of reference of camera
    # = lambda * [x',y',1, 1/lambda]
    xCamHom1 = T @ XHom
    # TO DO: Project points into normalized camera coordinates xCamHom (remove 4th row)
    # = lambda * [x',y',1]
    xCamHom = xCamHom1[0:3,:]
    # TO DO: Move points to image coordinates xImHom by applying intrinsic matrix
    # = lambda * [x,y,1]
    xImHom = K @ xCamHom
    # TO DO: Convert points back to Cartesian coordinates xImCart
    # x = (lambda * x) / lambda
    # y = (lambda * y) / lambda
    XImCart = xImHom[0:2,:] / np.tile([xImHom[2,:]],(2,1))
    return XImCart


In [4]:
def solveAXEqualsZero(A):
    # TO DO: Write this routine - it should solve Ah = 0   
    h = np.zeros(shape = [np.size(A),1])
    [U,L,Vt] = np.linalg.svd(A)
    V = np.transpose(Vt)
    h = V[:,-1]
    return h

In [8]:
# Goal of function is to estimate pose of plane relative to camera (extrinsic matrix)
# given points in image xImCart, points in world XCart and intrinsic matrix K

def estimatePlanePose(XImCart,XCart,K):

    # TO DO: replace this
    #T = 

    # TO DO: Convert Cartesian image points XImCart to homogeneous representation XImHom
    # by appending a row of 1 to the original point coordinates to form [x,y,1]
    XImHom = np.concatenate((XImCart, np.ones((1,XImCart.shape[1]))), axis=0)
    
    # TO DO: Convert image co-ordinates XImHom to normalized camera coordinates XCamHom    
    # multiply with inverse of intrinsic matrix
    XCamHom = np.linalg.inv(K) @ XImHom
    
    # TO DO: Estimate homography H mapping homogeneous (x,y) coordinates of positions
    # in real world to XCamHom (convert XCamHom to Cartesian, calculate the homography) -
    # use the routine you wrote for Practical 1B
    
    # Extract the u and v coordinates of Cartesian 3d points XCart
    XCart = XCart[0:2,:]
    
    # Convert XCart to homogeneous coordinates XCartHom
    # [u,v] to [u,v,1]
    XCartHom = np.concatenate((XCart, np.ones((1,XCart.shape[1]))), axis=0)
    
    # Then construct the matrix A, size (n_points,9) 
    n_points = np.shape(XCamHom)[1]
    A = np.zeros(shape = [n_points*2,9])
    for n in range(n_points):
        A[2*n,[0,1,2]] = 0
        A[2*n,[3,4,5]] = -XCartHom[:,n]
        A[2*n,[6,7,8]] = XCartHom[:,n] * XCamHom[1,n]
        A[2*n+1,[0,1,2]] = XCartHom[:,n]
        A[2*n+1,[3,4,5]] = 0
        A[2*n+1,[6,7,8]] = -XCartHom[:,n] * XCamHom[0,n]
    
    # Solve Ah = 0
    h = solveAXEqualsZero(A)
    # Reshape h into the matrix H, values of h go first into rows of H
    h_width = np.sqrt(np.size(h))
    h_width = int(h_width)
    H = np.reshape(h,[h_width,h_width])
     
    # TO DO: Estimate first two columns of rotation matrix R from the first two
    # columns of H using the SVD
    # SVD decomposition of the first two columns of H
    [U,L,Vt] = np.linalg.svd(H[:,0:2])
    R = np.zeros(shape=[h_width,h_width])
    # Replace L with [1,0;0,1;0,0] to form the first two columns of R
    l = np.array([[1,0],[0,1],[0,0]])
    R[:,0:2] = U @ l @ Vt

    # TO DO: Estimate the third column of the rotation matrix by taking the cross
    # product of the first two columns
    R[:,2] = np.cross(R[:,0],R[:,1])
        
    # TO DO: Check that the determinant of the rotation matrix is positive - if
    # not then multiply last column by -1.
    if np.linalg.det(R) <= 0 :
        R[:,-1] = - R[:,-1]
    
    # TO DO: Estimate the translation t by finding the appropriate scaling factor k
    # and applying it to the third colulmn of H
    # Find translation scaling factor between old and new values
    Lamb = H / R
    lamb = np.sum(Lamb[0:3,0:2]) / 6
    t = H[:,-1] / lamb
    
    # TO DO: Check whether t_z is negative - if it is then multiply t by -1 and
    # the first two columns of R by -1.
    if t[-1] < 0 :
        t = -t
        R[:,0:2] = -R[:,0:2]
    
            
    # TO DO: Assemble transformation into matrix form
    # append [Tx,Ty,Tz] to R as the last column
    # then append [0,0,0,1] as the last row to form the final transformation matrix
    t = np.reshape(t,[3,1])
    T = np.concatenate((R, t), axis=1)
    T = np.concatenate((T, np.array([[0,0,0,1]])), axis=0)
    
    return T 

# Once you have completed these functions, use them to estimate the transformation from the plane co-ordinate system to the camera co-ordinate system (i.e. the extrinsic matrix)

In [24]:
# We assume that the intrinsic camera matrix K is known and has values
K = np.array([[640, 0, 320],
             [0, 640, 240],
             [0, 0, 1]])

# We will assume an object co-ordinate system with the Z-axis pointing upwards and the origin
# in the centre of the plane. There are four known points on the plane with coordinates (mm):
XCart = np.array([[-100, -100,  100,  100, 0],
                  [-100,  100,  100, -100, 0],
                  [   0,    0,    0,    0, 0]])

# We assume the correct transformation from the plane co-ordinate system to the
# camera co-ordinate system (extrinsic matrix) is:
T = np.array([[0.9851,  -0.0492,  0.1619,  46.00],
             [-0.1623,  -0.5520,  0.8181,  70.00],
             [0.0490,  -0.8324, -0.5518,  500.89],
             [0,        0,       0,       1]])
  
# TO DO: Use the general pin-hole projective camera model discussed in the lectures to estimate 
# where the four points on the plane will appear in the image.  Fill in the
# details of the function "projectiveCamera" - body of function appears below:
XImCart = projectiveCamera(K,T,XCart)

# TO DO: Add noise (standard deviation of one pixel in each direction) to the pixel positions
# to simulate having to find these points in a noisy image. Store the results back in xImCart
XImCart[0,:] = XImCart[0,:] + np.random.normal(0,1,np.shape(XImCart[0,:]))
XImCart[1,:] = XImCart[1,:] + np.random.normal(0,1,np.shape(XImCart[0,:]))

# TO DO: Now we will take the image points and the known positions on the card and estimate  
# the extrinsic matrix using the algorithm discussed in the lecture.  Fill in the details of 
# the function "estimate plane pose"
TEst = estimatePlanePose(XImCart,XCart,K)
print(TEst)

# If you have got this correct, Test should closely resemble T above

[[ 9.85697964e-01 -4.81515155e-02  1.61495992e-01  4.62596500e+01]
 [-1.61296113e-01 -5.47167759e-01  8.21334893e-01  7.09770533e+01]
 [ 4.88168801e-02 -8.35636808e-01 -5.47108799e-01  5.05019303e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
